# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import SGD

# Loading Dataset
- Before we load the dataset, we need to define a transformation.
- A transformation is applied to the data before feeding it to the Neural Network.
- These transformations mostly consist of techniques used for data augmentation.
- In this example, we will not be using data augmentation.
- We will make use of transformation just to convert images to their tensor representations.

In [2]:
# Defining transformation to convert images to tensor
transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
])

In [9]:
# load our training data from the torchvision datasets module.
train_data = torchvision.datasets.MNIST(root = './19_train', train = True, download = True, transform = transform)

In [10]:
test_data = torchvision.datasets.MNIST(root = './19_test', train = False, download = True, transform = transform)

take out half of the test data as validation data for performing validation while training the model.

In [11]:
valid_data, test_data = random_split(test_data, [5000, 5000])

# Creating Data Loaders

In [12]:
train_dataloader = DataLoader(train_data, batch_size = 32)
valid_dataloader = DataLoader(valid_data, batch_size = 32)
test_dataloader = DataLoader(test_data, batch_size = 32)

# Creating Neural Network

In [13]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1=nn.Linear(784, 300)
        self.layer2=nn.Linear(300, 100)
        self.layer3=nn.Linear(100, 10)
    
    def forward(self, x):
        x=x.view(-1, 784)
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        return x

In [16]:
# create out Neural Network by creating an instance of the MyNet class
model = MyNet()
print(model)

MyNet(
  (layer1): Linear(in_features=784, out_features=300, bias=True)
  (layer2): Linear(in_features=300, out_features=100, bias=True)
  (layer3): Linear(in_features=100, out_features=10, bias=True)
)


# Creating Optimizer and Loss Functions

In [19]:
# create a Stochastic Gradient Descent(SGD) optimizer to optimize the parameters of our model and set its learning rate to 0.001.
optimizer = SGD(model.parameters(), lr = 0.001)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [20]:
# create a loss function to see how good or bad our model is performing at a given time.
loss_function = torch.nn.CrossEntropyLoss()
print(loss_function)

CrossEntropyLoss()


# Performing Training and Validation

train the model on the training dataset, while simultaneously measuring how the model performs on the the validation dataset.

In [21]:
for epoch in range(10):

    # Performing Training for each epoch
    training_loss = 0.
    model.train()

    # The training loop
    for batch in train_dataloader:
        optimizer.zero_grad()
        input, label = batch
        output = model(input)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()


    # Performing Validation for each epoch
    validation_loss = 0.
    model.eval()

    # The validation loop
    for batch in valid_dataloader:
        input, label = batch
        output = model(input)
        loss = loss_function(output, label)
        validation_loss += loss.item()

    # Calculating the average training and validation loss over epoch
    training_loss_avg = training_loss/len(train_dataloader)
    validation_loss_avg = validation_loss/len(valid_dataloader)

    # Printing average training and average validation losses
    print("Epoch: {}".format(epoch))
    print("Training loss: {}".format(training_loss_avg))
    print("Validation loss: {}".format(validation_loss_avg))

Epoch: 0
Training loss: 3.553650709025065
Validation loss: 2.4443936568156928
Epoch: 1
Training loss: 2.080358124860128
Validation loss: 1.4499497064359628
Epoch: 2
Training loss: 1.1282290950934093
Validation loss: 0.8742818574237216
Epoch: 3
Training loss: 0.7909886005083719
Validation loss: 0.6718631428518113
Epoch: 4
Training loss: 0.6476645072698594
Validation loss: 0.5691200978816695
Epoch: 5
Training loss: 0.5677690280357996
Validation loss: 0.506357674575915
Epoch: 6
Training loss: 0.5162254646341006
Validation loss: 0.46394608535204723
Epoch: 7
Training loss: 0.48007792270978294
Validation loss: 0.43342285418206716
Epoch: 8
Training loss: 0.453288878150781
Validation loss: 0.4104569557176274
Epoch: 9
Training loss: 0.4326092449426651
Validation loss: 0.39255966265110454


# Testing Accuracy

In [22]:
# Setting the number of correct predictions to 0
num_correct_pred = 0

# Running the model over test dataset and calculating total correct predictions
for batch in test_dataloader:
        input, label = batch
        output = model(input)
        _, predictions = torch.max(output.data, 1)
        num_correct_pred += (predictions == label).sum().item()

# Calculating the accuracy of model on test dataset
accuracy = num_correct_pred/(len(test_dataloader)*test_dataloader.batch_size)

print(accuracy)

0.8877388535031847


# Making Predictions

In [26]:
# Making Prediction
prediction = model(test_data[1][0])
print(prediction)

tensor([[6.8378e+00, 5.1838e+00, 5.7442e+00, 1.0463e+01, 6.0611e+00, 1.1388e+01,
         6.5693e+00, 5.1592e+00, 1.0250e+01, 7.4876e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.7052e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0965e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1314e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4714e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.5209e-02,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.1360e-02,
         0.0000e+00, 2.4796e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 6.9357e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.4562e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [28]:
print(prediction.argmax().item())
# Outputs- 5

5
